# OLS Analysis Using Full PSU dataset

In [1]:
#Import required packages
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
def format_date(df_date):
    """
    Splits Meeting Times and Dates into datetime objects where applicable using regex.
    """
    df_date['Days'] = df_date['Meeting_Times'].str.extract('([^\s]+)', expand=True)
    df_date['Start_Date'] = df_date['Meeting_Dates'].str.extract('([^\s]+)', expand=True)
    df_date['Year'] = df_date['Term'].astype(str).str.slice(0,4)
    df_date['Quarter'] = df_date['Term'].astype(str).str.slice(4,6)
    df_date['Term_Date'] = pd.to_datetime(df_date['Year'] + df_date['Quarter'], format='%Y%m')
    df_date['End_Date'] = df_date['Meeting_Dates'].str.extract('(?<=-)(.*)(?= )', expand=True)
    df_date['Start_Time'] = df_date['Meeting_Times'].str.extract('(?<= )(.*)(?=-)', expand=True)
    df_date['Start_Time'] = pd.to_datetime(df_date['Start_Time'], format='%H%M')
    df_date['End_Time'] = df_date['Meeting_Times'].str.extract('((?<=-).*$)', expand=True)
    df_date['End_Time'] = pd.to_datetime(df_date['End_Time'], format='%H%M')
    df_date['Duration_Hr'] = ((df_date['End_Time'] - df_date['Start_Time']).dt.seconds)/3600
    return df_date

def format_xlist(df_xl):
    """
    revises % capacity calculations by using Max Enrollment instead of room capacity.  
    """
    df_xl['Cap_Diff'] = np.where(df_xl['Xlst'] != '', 
                                   df_xl['Max_Enrl'].astype(int) - df_xl['Actual_Enrl'].astype(int), 
                                   df_xl['Room_Capacity'].astype(int) - df_xl['Actual_Enrl'].astype(int)) 
    df_xl = df_xl.loc[df_xl['Room_Capacity'].astype(int) < 999]

    return df_xl   
    

## Partitioning a dataset in training and test sets

In [24]:
pd.set_option('display.max_rows', None)  


df = pd.read_csv('data/PSU_master_classroom_91-17.csv', dtype={'Schedule': object, 'Schedule Desc': object})
df = df.fillna('')

df = format_date(df)
# Avoid classes that only occur on a single day
df = df.loc[df['Start_Date'] != df['End_Date']]

#terms = [199104, 199204, 199304, 199404, 199504, 199604, 199704, 199804, 199904, 200004, 200104, 200204, 200304, 200404, 200504, 200604, 200704, 200804, 200904, 201004, 201104, 201204, 201304, 201404, 201504, 201604]
#terms = [200604, 200704, 200804, 200904, 201004, 201104, 201204, 201304, 201404, 201504, 201604]
#df = df.loc[df['Term'].isin(terms)]
df = df.loc[df['Online Instruct Method'] != 'Fully Online']
#dept_lst = ['MTH', 'CH', 'BI', 'CE', 'CS', 'ECE', 'EMGT' ]
#df = df.loc[df['Dept'].isin(dept_lst)]

# Calculate number of days per week and treat Sunday condition
df['Days_Per_Week'] = df['Days'].str.len()
df['Room_Capacity'] = df['Room_Capacity'].apply(lambda x: x if (x != 'No Data Available') else 0)
df['Building'] = df['ROOM'].str.extract('([^\s]+)', expand=True)

df_cl = format_xlist(df)
df_cl['%_Empty'] = df_cl['Cap_Diff'].astype(float) / df_cl['Room_Capacity'].astype(float)

# Normalize the results
df_cl['%_Empty'] = df_cl['Actual_Enrl'].astype(np.float32)/df_cl['Room_Capacity'].astype(np.float32)
df_cl = df_cl.replace([np.inf, -np.inf], np.nan).dropna()
df_cl = df_cl.loc[df_cl['%_Empty'] != 0]

group_ops = ({'%_Empty' : 'mean',
              'Actual_Enrl' : 'mean',
             'Title' : 'max'})
df_cl = df_cl.groupby('Class', as_index=False).agg(group_ops)
df_cl = df_cl.loc[df_cl['Actual_Enrl'] >= 10]
df_cl = df_cl.loc[df_cl['%_Empty'] <= .2]


#df_cl = df_cl.sample(n = 75000)
print(df_cl.sort_values(by=(['%_Empty']), ascending=True))



C:\Users\kms22\AppData\Local\Continuum\Miniconda3\envs\data-science\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kms22\AppData\Local\Continuum\Miniconda3\envs\data-science\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


         Class                           Title   %_Empty  Actual_Enrl
1734   CS 447P               COMPUTER GRAPHICS  0.075000    21.000000
5532   PHE 417      ADAPTED PHYSICAL EDUCATION  0.097656    12.500000
5045   MUS 395                    CONCERT BAND  0.108628    10.244898
5194    PA 316  LEADERSHIP IN NEW STUDENT PROG  0.113537    26.000000
8334    WS 429        WOMEN IN THE VISUAL ARTS  0.115789    11.000000
4975   MUS 198             JAZZ VOCAL ENSEMBLE  0.128333    10.171429
4936  MUED 480                 KODALY: LEVEL I  0.131250    10.500000
4989   MUS 237                BRASS TECHNIQUES  0.132940    12.666667
940   BST 470U                     AFRICAN ART  0.133986    12.500000
5048   MUS 398             JAZZ VOCAL ENSEMBLE  0.134906    11.018868
2902   ESR 607                         SEMINAR  0.135320    12.350000
1953    EC 518    ECONOMICS DEPARTMENT SEMINAR  0.136299    10.384615
4994   MUS 262              HISTORY ROCK MUSIC  0.137547    14.333333
4054  INTL 372      

In [21]:
pd.set_option('display.max_rows', None)  


df = pd.read_csv('data/PSU_master_classroom_91-17.csv', dtype={'Schedule': object, 'Schedule Desc': object})
df = df.fillna('')

df = format_date(df)
# Avoid classes that only occur on a single day
df = df.loc[df['Start_Date'] != df['End_Date']]

#terms = [199104, 199204, 199304, 199404, 199504, 199604, 199704, 199804, 199904, 200004, 200104, 200204, 200304, 200404, 200504, 200604, 200704, 200804, 200904, 201004, 201104, 201204, 201304, 201404, 201504, 201604]
#terms = [200604, 200704, 200804, 200904, 201004, 201104, 201204, 201304, 201404, 201504, 201604]
#df = df.loc[df['Term'].isin(terms)]
df = df.loc[df['Online Instruct Method'] != 'Fully Online']
#dept_lst = ['MTH', 'CH', 'BI', 'CE', 'CS', 'ECE', 'EMGT' ]
#df = df.loc[df['Dept'].isin(dept_lst)]

# Calculate number of days per week and treat Sunday condition
df['Days_Per_Week'] = df['Days'].str.len()
df['Room_Capacity'] = df['Room_Capacity'].apply(lambda x: x if (x != 'No Data Available') else 0)
df['Building'] = df['ROOM'].str.extract('([^\s]+)', expand=True)

df_cl = format_xlist(df)
df_cl['%_Empty'] = df_cl['Cap_Diff'].astype(float) / df_cl['Room_Capacity'].astype(float)

# Normalize the results
df_cl['%_Empty'] = df_cl['Actual_Enrl'].astype(np.float32)/df_cl['Room_Capacity'].astype(np.float32)
df_cl = df_cl.replace([np.inf, -np.inf], np.nan).dropna()
df_cl = df_cl.loc[df_cl['%_Empty'] != 0]

group_ops = ({'%_Empty' : 'mean',
              'Actual_Enrl' : 'mean',
             'Title' : 'max'})
df_cl = df_cl.groupby('Class', as_index=False).agg(group_ops)
df_cl = df_cl.loc[df_cl['%_Empty'] >= .5]


#df_cl = df_cl.sample(n = 75000)
print(df_cl.sort_values(by=(['%_Empty']), ascending=False))



C:\Users\kms22\AppData\Local\Continuum\Miniconda3\envs\data-science\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kms22\AppData\Local\Continuum\Miniconda3\envs\data-science\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


          Class                           Title   %_Empty  Actual_Enrl
6184    SOC 396            RESEARCH METHODS LAB  8.071428    14.500000
516     ART 328            COMPUTER GRAPHICS II  6.691667    19.500000
468    ART 120L                 LAB FOR ART 120  5.234649    18.526316
469    ART 120R              STUDIO FOR ART 120  5.234649    18.526316
6809   SYSC 514                 SYSTEM DYNAMICS  3.508662    18.578947
6186   SOC 397L                LAB SOC RES METH  2.696908    17.189474
8218     WR 430                 DESKTOP PUBL II  2.481814    14.250000
6266    SOC 593            QUANTITATIVE METHODS  2.319173    13.000000
8197     WR 330            DESKTOP PUBLISHING I  2.222296    16.695652
272    ARCH 351              ARCH STRUCTURES II  2.012500    38.000000
3234     G 322L                    LAB FOR G322  1.766610    10.400000
3236     G 324L                   LAB FOR G 324  1.761905    16.619048
482     ART 228             COMPUTER GRAPHICS I  1.730556    22.916667
5355  